In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import warnings
warnings.filterwarnings('ignore')

from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, 

SyntaxError: trailing comma not allowed without surrounding parentheses (3643326121.py, line 21)

In [19]:
data=pd.read_csv('salarydata.csv')
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   education       32561 non-null  object
 3   education-num   32561 non-null  int64 
 4   marital-status  32561 non-null  object
 5   occupation      32561 non-null  object
 6   relationship    32561 non-null  object
 7   race            32561 non-null  object
 8   sex             32561 non-null  object
 9   capital-gain    32561 non-null  int64 
 10  capital-loss    32561 non-null  int64 
 11  hours-per-week  32561 non-null  int64 
 12  native-country  32561 non-null  object
 13  salary          32561 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


In [21]:
data.duplicated().sum()

3465

In [22]:
data.drop_duplicates(inplace=True)

In [23]:
data.isnull().sum()

age               0
workclass         0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
dtype: int64

In [24]:
data.shape

(29096, 14)

In [25]:
data.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week
count,29096.000000,29096.000000,29096.000000,29096.000000,29096.000000
mean,39.251340,10.102695,1197.802206,97.175179,40.637820
std,13.687157,2.645194,7778.225220,424.008232,12.735418
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,38.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [26]:
#DATA CLEANING
data.isin(['?']).sum(axis=0)

age                  0
workclass         1632
education            0
education-num        0
marital-status       0
occupation        1639
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     580
salary               0
dtype: int64

In [27]:
# code will replace the special character to nan and then drop the columns 
data['native-country'] = data['native-country'].replace('?',np.nan)
data['workclass'] = data['workclass'].replace('?',np.nan)
data['occupation'] = data['occupation'].replace('?',np.nan)
#dropping the NaN rows now 
data.dropna(how='any',inplace=True)

In [34]:
#find categorical variables

categorical = [var for var in data.columns if data[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

There are 9 categorical variables

The categorical variables are :

 ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'salary']


In [35]:
data[categorical].head()


,workclass,education,marital-status,occupation,relationship,race,sex,native-country,salary
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [36]:
data[categorical].isnull().sum()

workclass         0
education         0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
salary            0
dtype: int64

In [40]:
categorical = [var for var in data.columns if data[var].dtype=='O']
for v in categorical:
    print(data[v].value_counts())

Private             19214
Self-emp-not-inc     2431
Local-gov            2014
State-gov            1253
Self-emp-inc         1049
Federal-gov           929
Without-pay            14
Name: workclass, dtype: int64
HS-grad         8268
Some-college    5894
Bachelors       4503
Masters         1558
Assoc-voc       1256
Assoc-acdm       994
11th             941
10th             770
7th-8th          541
Prof-school      532
9th              449
Doctorate        364
12th             358
5th-6th          283
1st-4th          149
Preschool         44
Name: education, dtype: int64
Married-civ-spouse       12371
Never-married             8388
Divorced                  4009
Separated                  928
Widowed                    817
Married-spouse-absent      370
Married-AF-spouse           21
Name: marital-status, dtype: int64
Prof-specialty       3783
Exec-managerial      3646
Adm-clerical         3291
Craft-repair         3229
Sales                3205
Other-service        2914
Machine-op-ins

In [41]:
data.occupation.unique()

array(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners',
       'Prof-specialty', 'Other-service', 'Sales', 'Transport-moving',
       'Farming-fishing', 'Machine-op-inspct', 'Tech-support',
       'Craft-repair', 'Protective-serv', 'Armed-Forces',
       'Priv-house-serv'], dtype=object)

In [42]:
data.occupation.value_counts()

Prof-specialty       3783
Exec-managerial      3646
Adm-clerical         3291
Craft-repair         3229
Sales                3205
Other-service        2914
Machine-op-inspct    1666
Transport-moving     1420
Handlers-cleaners    1159
Farming-fishing       957
Tech-support          858
Protective-serv       626
Priv-house-serv       141
Armed-Forces            9
Name: occupation, dtype: int64

In [47]:

numerical = [var for var in data.columns if data[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :\n\n', numerical)

There are 5 numerical variables

The numerical variables are :

 ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


In [48]:
data[numerical].head()

,age,education-num,capital-gain,capital-loss,hours-per-week
0,39,13,2174,0,40
1,50,13,0,0,13
2,38,9,0,0,40
3,53,7,0,0,40
4,28,13,0,0,40


In [49]:
data[numerical].isnull().sum()

age               0
education-num     0
capital-gain      0
capital-loss      0
hours-per-week    0
dtype: int64